In [3]:
from moviepy.editor import *
from os import listdir
from os.path import isfile, join
import random

In [2]:
#Załadowanie wszystkich klipów z folderu, warto znać ich kolejność
clip_path = "my_street_film/clips/"
def load_files(clip_path):
    files = [f for f in listdir(clip_path)]
    files = [clip_path + f for f in files]
    i=0
    for n in files:
        print(i,":",n)
        i+=1
    multi_clips = [n for n in files]
    return multi_clips

multi_clips = load_files(clip_path)
multi_music = load_files("my_street_film/music/")


0 : my_street_film/clips/MVI_7176.AVI
1 : my_street_film/clips/MVI_7177.AVI
2 : my_street_film/clips/MVI_7179.AVI
3 : my_street_film/clips/MVI_7182.AVI
4 : my_street_film/clips/MVI_7183.AVI
5 : my_street_film/clips/MVI_7184.AVI
6 : my_street_film/clips/MVI_7185.AVI
7 : my_street_film/clips/MVI_7186.AVI
8 : my_street_film/clips/MVI_7188.AVI
9 : my_street_film/clips/MVI_7190.AVI
10 : my_street_film/clips/MVI_7191.AVI
11 : my_street_film/clips/MVI_7197.AVI
12 : my_street_film/clips/MVI_7198.AVI
13 : my_street_film/clips/MVI_7202.AVI
14 : my_street_film/clips/MVI_7205.AVI
0 : my_street_film/music/Dream Lagoon - Chris Haugen.mp3
1 : my_street_film/music/Spring Migration - The Great North Sound Society.mp3
2 : my_street_film/music/The Crows Did It - Nathan Moore.mp3


In [5]:
def randomize_order(number_of_used_clips, number_of_reps, multi_clips):
#    number_of_reps = 2 #ile razy dany klip może być powtórzony dodatkowo
    
    if number_of_used_clips == None:
        number_of_used_clips = len(multi_clips)
    if number_of_used_clips > len(multi_clips):
        raise Exception("To big number of clips, should be:", len(multi_clips))
        
    order_clips = []

    final_number_of_clips =  number_of_used_clips + number_of_reps

    jackpot =[n for n in range(0,len(multi_clips))]
    super_jackpot = jackpot.copy()

    while True:
        if len(order_clips) == number_of_used_clips:
            for n in range(0,number_of_reps):
                super_random = random.choice(super_jackpot)
                order_clips.append(super_random)
            break

        randomize = random.choice(jackpot)

        if randomize in order_clips:
            jackpot.remove(randomize)
            continue

        order_clips.append(randomize)
    
    return order_clips

def arrange_clips(min_time, max_time, multi_clips, random_order):
    video_clips =[]
    end_times = []
    end_times.append(0)
    
    for n in random_order:
        clip = VideoFileClip(multi_clips[n])
        clip_time = clip.duration
        rand_time = round(random.uniform(min_time,max_time),2)
        if rand_time > clip_time:
            rand_time = clip_time
        rand_start = clip_time - rand_time
        rand_start = round(random.uniform(0,rand_start),2)
        clip = clip.subclip(rand_start,rand_start+rand_time)
        clip = clip.set_start(end_times[len(end_times)-1])
        end_times.append(clip.end)
        video_clips.append(clip)
        
    return video_clips


def arrange_music(multi_music, random_order, generated_clips, iscut, ismusic, fadetime):
    #iscut daje losowy przydzial audio
    clips_time = sum(map(float,[f.duration for f in generated_clips]))
    all_music_time = sum([AudioFileClip(f).duration for f in multi_music])
#    clips_music = [f.audio for f in generated_clips]
    clips_music = []
#    print(clips_music)
    print(clips_time)

    end_times = []
    end_times.append(0)
    
#    if isaudio == False:
#        clips_music = clips_music.clear()
        
    if ismusic == False:
        return clips_music
    
    if iscut:
        loop_counter = 0
        for n in random_order:
            music = AudioFileClip(multi_music[n])
            if loop_counter + 1 >= len(random_order):
                print("end time: ", clips_time)
                music = music.set_start(end_times[len(end_times)-1])
                music = music.set_end(clips_time)
                music = music.audio_fadein(fadetime).audio_fadeout(fadetime)
                print("end:",music.start, music.end)
            else:
                randomize = round(random.uniform(end_times[len(end_times)-1],clips_time), 3)
                music = music.set_start(end_times[len(end_times)-1])
                music = music.set_end(randomize)
                music = music.audio_fadein(fadetime).audio_fadeout(fadetime)
                print("ok:",music.start, music.end)
#            print(music)
            clips_music.append(music)
            end_times.append(music.end)
            loop_counter += 1
    else:
        for n in random_order:
            music = AudioFileClip(multi_music[n])
            if music.duration > (clips_time-end_times[len(end_times)-1]):
                music = music.set_start(end_times[len(end_times)-1])
                music = music.set_end(clips_time)
                music = music.audio_fadein(fadetime).audio_fadeout(fadetime)
                clips_music.append(music)
                break
            else:
                music = music.set_start(end_times[len(end_times)-1])
                music = music.set_start(end_times[len(end_times)-1])
                clips_music.append(music)
                end_times.append(music.end)
    return clips_music

random_clips_order = randomize_order(None,2, multi_clips)
generated_clips = arrange_clips(1,4,multi_clips,random_clips_order)

random_music_order = randomize_order(None,0, multi_music)
generated_music = arrange_music(multi_music, random_music_order, generated_clips, True, True, 1.5)

#print(random_music_order)

final = CompositeVideoClip(generated_clips)
final_audio = CompositeAudioClip(generated_music)
final = final.set_audio(final_audio)
final.write_videofile("output/random_film4.mp4")
#print(randomize_order)
#print(generate_clips)


    

42.42000000000001
ok: 0 9.811
ok: 9.811 29.982
end time:  42.42000000000001
end: 29.982 42.42000000000001


chunk:   3%|█▉                                                             | 28/936 [00:00<00:03, 269.36it/s, now=None]

Moviepy - Building video output/random_film4.mp4.
MoviePy - Writing audio in random_film4TEMP_MPY_wvf_snd.mp3


t:   0%|▎                                                                   | 6/1273 [00:00<00:21, 59.45it/s, now=None]

MoviePy - Done.
Moviepy - Writing video output/random_film4.mp4



Moviepy - Done !
Moviepy - video ready output/random_film4.mp4
